# Tutoriel avec ChatGPT

Ce tutoriel a été créé en collaboration avec ChatGPT pour aider à comprendre et à utiliser les modèles de diffusion en intelligence artificielle générative.

Il se base notamment sur le livre "Generative Deep Learning" de David Foster (voir le [notebook associé](https://github.com/tomboulier/Generative_Deep_Learning_2nd_Edition/blob/main/notebooks/08_diffusion/01_ddm/ddm.ipynb) pour plus de détails), qui lui-même s'inspire du tutoriel [Denoising Diffusion Implicit Models](https://keras.io/examples/generative/ddim/) d'András Béres. Je voulais les adapter pour qu'ils utilisent les données de [MedMNIST](https://medmnist.com/).

# Chargement du jeu de données

Nous allons charger le jeu de données MedMNIST pour l'utiliser dans notre tutoriel. En particulier, nous prendrons les radiographies thoraciques (ChestMNIST).

In [ ]:
from medmnist import ChestMNIST

train_dataset = ChestMNIST(split="train", download=True)